In [1]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('There are %d GPU(s) available' % torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print('No GPU available, CPU instaed')

device

There are 1 GPU(s) available
NVIDIA A100 80GB PCIe


device(type='cuda')

In [2]:
!pip install transformers
!pip install wget

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import pandas as pd
import numpy as np
from copy import deepcopy
import re
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

import torch
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig

import numpy as np
import datetime
import time
import random
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, \
                            precision_score, recall_score, accuracy_score, roc_auc_score, average_precision_score

In [4]:
incident_1 = pd.read_excel('/opt/share/DAOzerova/nlp/data/Инциденты 01-03.2023.xlsx')
incident_2 = pd.read_excel('/opt/share/DAOzerova/nlp/data/Инциденты 04-06.2023.xlsx')
incident_3 = pd.read_excel('/opt/share/DAOzerova/nlp/data/Инциденты 07-09.2023.xlsx')
incident_4 = pd.read_excel('/opt/share/DAOzerova/nlp/data/Инциденты 10-12.2023.xlsx')
incident_5 = pd.read_excel('/opt/share/DAOzerova/nlp/data/ЕИ_2024_01-04.xlsx')

incident_1 = incident_1.rename(columns={"Когда создано": "Дата открытия**", "Ответственная группа": "ID1 я Группа", "Назначено на группу": "Группа", "Номер": "Номер*"})
incident_2 = incident_2.rename(columns={"Когда создано": "Дата открытия**", "Ответственная группа": "ID1 я Группа", "Назначено на группу": "Группа", "Номер": "Номер*"})
incident_3 = incident_3.rename(columns={"Когда создано": "Дата открытия**", "Ответственная группа": "ID1 я Группа", "Назначено на группу": "Группа", "Номер": "Номер*"})
incident_4 = incident_4.rename(columns={"Когда создано": "Дата открытия**", "Ответственная группа": "ID1 я Группа", "Назначено на группу": "Группа", "Номер": "Номер*"})
incident_5 = incident_5.rename(columns={"Когда создано": "Дата открытия**", "Ответственная группа": "ID1 я Группа", "Назначено на группу": "Группа", "Номер": "Номер*"})

data = pd.concat([incident_1, incident_2, incident_3, incident_4, incident_5], ignore_index=True)
data.head()

,Номер*,Дата открытия**,Решено в,Услуга,Модель запроса,Тема,Описание,ID1 я Группа,Группа,Код закрытия,Информация о закрытии
0,INC0299994,2023-03-31 23:59:05,2023-04-03 11:40:36,Сопровождение МТС Банк Premium (Премиум),МТС Банк Premium Просьба Персональному менедже...,Зибель. [Тип запроса]: Просьба Персональному м...,[Тип запроса]: Просьба Персональному менеджеру...,PMPremium,PMPremium,Решён полностью,ПМ Сбитинков Антон Анатольевич
1,INC0299993,2023-03-31 23:53:49,2023-04-01 00:23:36,Сопровождение МТС Банк,МТС Банк Не приходит СМС при авторизации,Зибель. [Тип запроса]: Во время авторизации не...,[Тип запроса]: Во время авторизации не приходи...,Отдел поддержки карточных технологий,Отдел поддержки карточных технологий,Решён полностью,Мы получили от оператора связи статус доставки...
2,INC0299992,2023-03-31 23:53:04,2023-04-18 18:58:55,Сопровождение СБП,СБП Проблемы при пополнении из другого Банка,Зибель. просьба помочь исправить у клиента в п...,просьба помочь исправить у клиента в приложени...,SBPSUPPORT,SBPSUPPORT,Известная ошибка,Клиент подключен к СБП. Переводы проходят успе...
3,INC0299991,2023-03-31 23:51:01,2023-04-05 12:03:08,Обеспечение переводов по номеру телефона в МТС...,МТС Банк После ввода реквизитов и нажатию на к...,Зибель. [Тип запроса]: Проблема с переводом по...,[Тип запроса]: Проблема с переводом по реквизи...,Поддержка FTB,Приложение МТС Банк,Решён полностью,В связи с введением ограничений на стороне про...
4,INC0299990,2023-03-31 23:48:09,2023-04-03 06:19:15,Обеспечение исправления некорректной информаци...,МТС Банк Некорректный баланс по картам клиента,Зибель. [Тип запроса]: Неверный баланс по кред...,[Тип запроса]: Неверный баланс по кредитной ка...,Корректировки баланса,Корректировки баланса,Решён обходным решением,Закрыто по результатам сверки счетов. Если бал...


In [5]:
groups_to_drop = [
    'Первая линия Сервисдеск', '1ЛТП анализ процессов',  
    'MB. SalesTools', 'CreditCards', 'NewHistory', 
    'Отдел контроля рисков', 'КЦ ВП Stream PFM'
]

df = data.copy()[~data['Группа'].isin(groups_to_drop)].reset_index(drop=True)

df.loc[:, ['Услуга', 'Модель запроса', 'ID1 я Группа']] =\
df.loc[:, ['Услуга', 'Модель запроса', 'ID1 я Группа']].fillna('')

df.shape

(481096, 11)

In [6]:
display(df['Номер*'].value_counts()[:5])

df.drop_duplicates(subset=['Номер*'], inplace=True, ignore_index=True)
df.drop_duplicates(subset=['Описание'], inplace=True, ignore_index=True)

df = df.dropna(subset=['Описание']).reset_index(drop=True)

display(df['Номер*'].value_counts()[:5])

INC0299994    1
INC0650940    1
INC0650924    1
INC0650927    1
INC0650928    1
Name: Номер*, dtype: int64

INC0299994    1
INC0650197    1
INC0650199    1
INC0650200    1
INC0650202    1
Name: Номер*, dtype: int64

In [7]:
df.shape

(474570, 11)

## Preprocesing ##

In [8]:
from collections import Counter

def drop_duplicates(txt):
    try:
        txt = re.split('\. |: |\n', txt)
        counter_txt = Counter(txt)
        if len(txt) < 2: return ''
        i = len(txt)-2
        while counter_txt[txt[i]] > 1:
            i-=1
    except:
        return ''
    if i==len(txt)-2:
        return " ".join(txt)
    return " ".join(txt[:i+1])

def replace_numbers(txt):
    client_pattern = r"\[ФИО клиента] \b[А-ЯЁ]+ [А-ЯЁ]+ [А-ЯЁ]+\b" 
    initiator_pattern = r'(\[Инициатор] [А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+)'
    phone_pattern = r'\b(\+)?([78]\d{10})\b'
    date_pattern = r'\d{1,2}\.\d{1,2}\.\d{2,4}'
    time_pattern = r'(\d{1,2}\:)?\d{1,2}\:\d{1,2}'
    passport_pattern = r'\d{4} \d{6}'
    nums_pattern = r'(\d{1,2} )?\d+(\,\d{1,2})?'
    trash_pattern = r'УКАЖИ'
    
    new_text = re.sub(client_pattern, ' client_number ', txt)
    new_text = re.sub(initiator_pattern, ' initiator_number ', new_text)
    new_text = re.sub(phone_pattern, ' phone_number ', new_text)
    new_text = re.sub(date_pattern, 'date ', new_text)
    new_text = re.sub(time_pattern, 'time ', new_text)
    new_text = re.sub(passport_pattern, 'passport ', new_text)
    new_text = re.sub(nums_pattern, 'number ', new_text)
    new_text = re.sub(trash_pattern, '', new_text)
    
    new_text = re.sub(r'[^\w\s]','', new_text).lower()
    new_text = re.sub(r'[\\]',' ', new_text).lower()
    return new_text

def full_describe_request(txt, add_feature_txt, add_txt):
    return txt + ' [' + str(add_feature_txt) + ' ]:' + str(add_txt)

def parse(series, features, text_col = 'Описание'):
    txt = '' if not isinstance(series[text_col], str) else series[text_col] 
    uniq_txt = drop_duplicates(txt)
    
    for feature in features:
        uniq_txt = full_describe_request(uniq_txt, feature, series[feature])
    txt = replace_numbers(uniq_txt)
    return txt.strip("\n")

In [9]:
df['Описание заявки'] = df.progress_apply(
    parse, 
    args = (
     ['Услуга', 'Модель запроса', 'ID1 я Группа'],
    ), 
    axis = 1,
)

df.drop_duplicates(subset=['Описание заявки'], inplace=True, ignore_index=True)

request_txts = df['Описание заявки']

print(df['Описание заявки'])

  0%|          | 0/474570 [00:00<?, ?it/s]

0         тип запроса просьба персональному менеджеру св...
1         тип запроса во время авторизации не приходит с...
2         просьба помочь исправить у клиента в приложени...
3         тип запроса проблема с переводом по реквизитам...
4         тип запроса неверный баланс по кредитной карте...
                                ...                        
460668     описание проблемы клиента у клиента некоррект...
460669     описание проблемы клиента не поступают запрос...
460670     описание проблемы клиента у клиента при подпи...
460671     описание проблемы клиента  телефон отправител...
460672     описание проблемы клиента перевод по сбп не п...
Name: Описание заявки, Length: 460673, dtype: object


## TARGET ##

In [10]:
df['TARGET'] = df['Группа']
df.loc[df['Группа'] == df['ID1 я Группа'], 'TARGET'] = 'Правильная заявка'
df.loc[df['TARGET'].isin(df['TARGET'].value_counts()[15:].index), 'TARGET'] = 'Другое: неправильная заявка'
df['TARGET'].value_counts()

Правильная заявка                       381047
Другое: неправильная заявка              10297
Авторизация. Неактуальная                 9415
Поддержка ПЦ ЭК                           8712
Приложение МТС Банк                       8317
Поддержка бизнес процессов                7818
Отдел активно-пассивных операций          6321
Отдел поддержки карточных технологий      5452
MB.oplata_dbo                             3928
Платёжный Хаб                             3848
DebitCardsSupport                         3140
Переводы между своими счетами             3110
Поддержка СДП.Платежи                     3015
МТС Деньги ВЕБ                            2709
Антифрод                                  2691
Возврат средств                            853
Name: TARGET, dtype: int64

In [11]:
df[['Услуга', 'Модель запроса', 'ID1 я Группа']]

,Услуга,Модель запроса,ID1 я Группа
0,Сопровождение МТС Банк Premium (Премиум),МТС Банк Premium Просьба Персональному менедже...,PMPremium
1,Сопровождение МТС Банк,МТС Банк Не приходит СМС при авторизации,Отдел поддержки карточных технологий
2,Сопровождение СБП,СБП Проблемы при пополнении из другого Банка,SBPSUPPORT
3,Обеспечение переводов по номеру телефона в МТС...,МТС Банк После ввода реквизитов и нажатию на к...,Поддержка FTB
4,Обеспечение исправления некорректной информаци...,МТС Банк Некорректный баланс по картам клиента,Корректировки баланса
...,...,...,...
460668,Обеспечение открытия продуктов в МТС Банк,МТС Банк Проблемы с дебетовыми картами,Приложение МТС Банк
460669,Сопровождение СБП,СБП Проблемы при отправке перевода в другой Банк,SBPSUPPORT
460670,Сопровождение CRM Siebel,CRM Siebel Проблема оформления и выдачи карты ...,Поддержка бизнес процессов
460671,Сопровождение СБП,СБП Проблемы при отправке перевода в другой Банк,SBPSUPPORT


In [12]:
if 'Описание заявки' not in df.columns:
    df = df.join(request_txts.rename('Описание заявки'))
    
df[['Номер*', 'Описание заявки']].to_parquet("requests_bert.parquet", index=False, compression='gzip')
request_txts = pd.read_parquet("requests_bert.parquet").reset_index(drop=True)

if 'Описание заявки' in df.columns:
    df.drop(columns=['Описание заявки'], inplace=True)
    
df = df.merge(request_txts, on = 'Номер*', how = 'left')
df[['Описание заявки', 'Тема', 'Услуга', 'Модель запроса', 'ID1 я Группа']]

,Описание заявки,Тема,Услуга,Модель запроса,ID1 я Группа
0,тип запроса просьба персональному менеджеру св...,Зибель. [Тип запроса]: Просьба Персональному м...,Сопровождение МТС Банк Premium (Премиум),МТС Банк Premium Просьба Персональному менедже...,PMPremium
1,тип запроса во время авторизации не приходит с...,Зибель. [Тип запроса]: Во время авторизации не...,Сопровождение МТС Банк,МТС Банк Не приходит СМС при авторизации,Отдел поддержки карточных технологий
2,просьба помочь исправить у клиента в приложени...,Зибель. просьба помочь исправить у клиента в п...,Сопровождение СБП,СБП Проблемы при пополнении из другого Банка,SBPSUPPORT
3,тип запроса проблема с переводом по реквизитам...,Зибель. [Тип запроса]: Проблема с переводом по...,Обеспечение переводов по номеру телефона в МТС...,МТС Банк После ввода реквизитов и нажатию на к...,Поддержка FTB
4,тип запроса неверный баланс по кредитной карте...,Зибель. [Тип запроса]: Неверный баланс по кред...,Обеспечение исправления некорректной информаци...,МТС Банк Некорректный баланс по картам клиента,Корректировки баланса
...,...,...,...,...,...
460668,описание проблемы клиента у клиента некоррект...,Зибель. УКАЖИ\n[Описание проблемы клиента]: У ...,Обеспечение открытия продуктов в МТС Банк,МТС Банк Проблемы с дебетовыми картами,Приложение МТС Банк
460669,описание проблемы клиента не поступают запрос...,Зибель. УКАЖИ\n[Описание проблемы клиента]: не...,Сопровождение СБП,СБП Проблемы при отправке перевода в другой Банк,SBPSUPPORT
460670,описание проблемы клиента у клиента при подпи...,Зибель. УКАЖИ\n[Описание проблемы клиента]: у ...,Сопровождение CRM Siebel,CRM Siebel Проблема оформления и выдачи карты ...,Поддержка бизнес процессов
460671,описание проблемы клиента телефон отправител...,Зибель. УКАЖИ\n[Описание проблемы клиента]: \n...,Сопровождение СБП,СБП Проблемы при отправке перевода в другой Банк,SBPSUPPORT


In [13]:
le = LabelEncoder()
le.fit(df['TARGET'].value_counts().index)
df['TARGET_NUM'] = le.transform(df['TARGET'])

df['TARGET_BINARY'] = df['TARGET_NUM'].eq(14).astype(int)

df['TARGET_NUM'], df['TARGET_BINARY'], list(le.classes_), len(le.classes_)

(0         14
 1         14
 2         14
 3         15
 4         14
           ..
 460668    14
 460669    14
 460670    14
 460671    14
 460672    14
 Name: TARGET_NUM, Length: 460673, dtype: int64,
 0         1
 1         1
 2         1
 3         0
 4         1
          ..
 460668    1
 460669    1
 460670    1
 460671    1
 460672    1
 Name: TARGET_BINARY, Length: 460673, dtype: int64,
 ['DebitCardsSupport',
  'MB.oplata_dbo',
  'Авторизация. Неактуальная',
  'Антифрод',
  'Возврат средств',
  'Другое: неправильная заявка',
  'МТС Деньги ВЕБ',
  'Отдел активно-пассивных операций',
  'Отдел поддержки карточных технологий',
  'Переводы между своими счетами',
  'Платёжный Хаб',
  'Поддержка ПЦ ЭК',
  'Поддержка СДП.Платежи',
  'Поддержка бизнес процессов',
  'Правильная заявка',
  'Приложение МТС Банк'],
 16)

In [14]:
sentences= df['Описание заявки'].values
labels = df['TARGET_NUM'].values

In [17]:
df[['Описание заявки','TARGET_NUM']].to_parquet('df.parquet')

In [68]:
X_train = pd.read_parquet('data/x_train.parquet')
y_train = pd.read_parquet('data/y_train.parquet')
X_test = pd.read_parquet('data/x_test.parquet')
y_test = pd.read_parquet('data/y_test.parquet')

In [7]:
#X_train['Описание заявки'].values

array(['мр сибирь омск платеж  электронный кошелек числиться операция  отклонять банк банк сообщать    денежный средство  отправлять    банк  возвращаться  выписка  мочь предоставлять  банк скопировать полностью строка  рм платеж  электронный кошелек   время  дата событие  number number  номер транзакция  number  сумма перевод  number  способ перевод выбирать значение  мтс  источник перевод выбирать значение  карта  банк  направление перевод выбирать значение  баланс телефон  баланс  баланс  первый number   последний number  цифра номер карта  который  попытка перевод  number xxxxxxnumber  пошаговый действие  оплата   мтс реквизит получатель платеж   phone_number  номер абонент  phone_number  imsi number  номер simкарты number  категория надежность категория надежность number  биллинговый группа мобильный связь стандартный название биллингаасра foris услуга  тп приложение мтс банк инициатор remedy мтс тема мтс деньги непоступление платеж услуга обеспечение перевод  мтс банк модель запр

In [8]:
#X_train['Описание заявки'].shape, y_train['TARGET_NUM'].shape

((308471,), (308471,))

In [6]:
sentences= X_train['Описание заявки'].values
labels = y_train['TARGET_NUM'].values

# DATA prepare

In [18]:
#%history

In [7]:
import os

tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/sbert_large_nlu_ru", 
    do_lower_case=True,
)
    
replace_tokens = {
        '[unused1]': 'мтс',
        '[unused2]': 'client_number',
        '[unused3]': 'initiator_number',
        '[unused4]': 'phone_number',
        '[unused5]': 'date',
        '[unused6]': 'passport',
        '[unused7]': 'number',
        '[unused8]': 'premium',
}

vocab = tokenizer.get_vocab()
    
for old_token, new_token in replace_tokens.items():
    idx = vocab[old_token]
    del vocab[old_token]
    vocab[new_token] = idx
    
# Тут надо отсортировать, иначе далее при инициализации токенайзера порядок ломается
vocab = pd.Series(vocab).sort_values()
vocab = vocab.index.tolist()
    
with open('vocab.tmp', 'w', encoding = 'utf-8') as tmp_vocab_file:
    tmp_vocab_file.write('\n'.join(vocab))
        
tokenizer = AutoTokenizer.from_pretrained(
    "ai-forever/sbert_large_nlu_ru", 
    vocab_file = 'vocab.tmp', do_lower_case=True, do_basic_tokenize=False
)
    
os.remove('vocab.tmp')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
print('original', sentences[0])
print('tokenised', tokenizer.tokenize(sentences[0], add_special_tokens=True))
print('toden IDs', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0], add_special_tokens=True)))

original мр сибирь омск платеж  электронный кошелек числиться операция  отклонять банк банк сообщать    денежный средство  отправлять    банк  возвращаться  выписка  мочь предоставлять  банк скопировать полностью строка  рм платеж  электронный кошелек   время  дата событие  number number  номер транзакция  number  сумма перевод  number  способ перевод выбирать значение  мтс  источник перевод выбирать значение  карта  банк  направление перевод выбирать значение  баланс телефон  баланс  баланс  первый number   последний number  цифра номер карта  который  попытка перевод  number xxxxxxnumber  пошаговый действие  оплата   мтс реквизит получатель платеж   phone_number  номер абонент  phone_number  imsi number  номер simкарты number  категория надежность категория надежность number  биллинговый группа мобильный связь стандартный название биллингаасра foris услуга  тп приложение мтс банк инициатор remedy мтс тема мтс деньги непоступление платеж услуга обеспечение перевод  мтс банк модель зап

In [9]:
len(tokenizer.encode(sentences[0], add_special_tokens=True))

190

In [10]:
len(sentences)

308471

In [11]:
max_len = 0
input_ids_all = []

for sent in tqdm(sentences):
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    input_ids_all.append(input_ids)

max_len

  0%|          | 0/308471 [00:00<?, ?it/s]

6982

In [12]:
input_ids = []
attention_masks = []

for sent in tqdm(sentences):
  encoded_dict = tokenizer.encode_plus(
      sent,
      add_special_tokens=True,
      max_length=256,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
  )
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print(sentences[0])
print(input_ids[0])

  0%|          | 0/308471 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


мр сибирь омск платеж  электронный кошелек числиться операция  отклонять банк банк сообщать    денежный средство  отправлять    банк  возвращаться  выписка  мочь предоставлять  банк скопировать полностью строка  рм платеж  электронный кошелек   время  дата событие  number number  номер транзакция  number  сумма перевод  number  способ перевод выбирать значение  мтс  источник перевод выбирать значение  карта  банк  направление перевод выбирать значение  баланс телефон  баланс  баланс  первый number   последний number  цифра номер карта  который  попытка перевод  number xxxxxxnumber  пошаговый действие  оплата   мтс реквизит получатель платеж   phone_number  номер абонент  phone_number  imsi number  номер simкарты number  категория надежность категория надежность number  биллинговый группа мобильный связь стандартный название биллингаасра foris услуга  тп приложение мтс банк инициатор remedy мтс тема мтс деньги непоступление платеж услуга обеспечение перевод  мтс банк модель запрос мтс б

In [13]:
len(input_ids), len(attention_masks), len(labels)

(308471, 308471, 308471)

In [14]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size

train_data, val_data = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

293,047 training samples
15,424 validation samples


In [68]:
'''
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = int(0.5 * (len(dataset) - train_size))
test_size = len(dataset) - train_size - val_size

train_data, val_data = random_split(dataset, [train_size, val_size + test_size])
val_data, test_data = random_split(val_data, [val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} testing samples'.format(test_size))
'''

368,538 training samples
46,067 validation samples
46,068 testing samples


In [15]:
batch_size = 32

train_dataloader = DataLoader(
    train_data,
    sampler = RandomSampler(train_data),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_data,
    sampler = SequentialSampler(val_data),
    batch_size=batch_size
)

# BERT prepare

In [16]:
import torch

t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

a, b = torch.cuda.mem_get_info()
print(a / 1024 / 1024 / 1024)
print(b / 1024 / 1024 / 1024)

56.23052978515625
79.138427734375


In [17]:
#model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

model = BertForSequenceClassification.from_pretrained(
    "ai-forever/sbert_large_nlu_ru", #'bert-base-uncased',
    num_labels = 16,
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=

In [18]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                  )


/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
epochs = 3

total_steps = len(train_dataloader) * epochs
scheduler =  get_linear_schedule_with_warmup(optimizer,
                                             num_warmup_steps = 0,
                                             num_training_steps = total_steps)

In [ ]:
val_data.indices, train_data.indices

# Training loop

**Training:**
* Распаковать обучающие данные и лейблы
* Загрузить данные на GPU для ускорения
* Занулить градиенты с предыдущего шага
* Forward pass (скормить данные в нейросеть и пробросить их вперед)
* Backward pass (back propagation - посчитать градиенты по всем параметрам с помощью обратного распространения ошибки)
* Обновить параметры с помощью optimizer.step()
* Посчитать статистики, чтобы следить за обучением
⛅

**Evaluation:**
* Распаковать валидационные данные и лейблы
* Загрузить данные на GPU для ускорения
* Forward pass (скормить данные в нейросеть и пробросить вперед)
* Посчиатть loss и статистики для валиданционных данных, чтобы следить за обучением



In [20]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

a, b = torch.cuda.mem_get_info()
print(a / 1024 / 1024 / 1024)
print(b / 1024 / 1024 / 1024)

54.13677978515625
79.138427734375


In [21]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

def  format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds=elapsed_rounded))


In [22]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
  print('')
  print('======= EPOCH {:} / {:} ======'.format(epoch_i + 1, epochs))
  print('Training...')

  t0 = time.time()
  total_train_loss = 0
  model.train()

  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      #Report progress
      print('   Batch {0:>5} of {1:>5}. Elapsed: {2:} Loss: {3:.4f}  Average_Train_Loss: {4:.4f}'.format(step, len(train_data), elapsed, res['loss'], total_train_loss/(batch_size * (step + 1))))

    #batch  contains 3 pytorch tensors: [0] input ids, [1] aattention masks, [2] labels
    #print(batch[0])
    #print(batch[1])
    #print(batch[2])
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    model.zero_grad()
    #Forward pass
    res = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels)
    loss = res['loss']
    logits = res['logits'] #вероятности классов для батча

    total_train_loss += loss.item()
    loss.backward()
    #нормы градиентов обрезаем до 1.0, чтобы предотвратить проблему взрывающихся градиентов
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step() #обновление весов
    scheduler.step()

    avg_train_loss = total_train_loss/len(train_dataloader)
    train_time = format_time(time.time() - t0)

  print(' Average training loss: {0:.2f}'.format(avg_train_loss))
  print(' Training epoch took: {:}'.format(train_time))


  #Validation
  print("Running Validation...")

  t0 = time.time()
  model.eval()
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in validation_dataloader:
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():
          # Forward pass, calculate logit predictions.
          result = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels,
                        return_dict=True)
      loss = result.loss
      logits = result.logits
      total_eval_loss += loss.item()
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      total_eval_accuracy += flat_accuracy(logits, label_ids)

  avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

  avg_val_loss = total_eval_loss / len(validation_dataloader)
  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

  training_stats.append(
      {
          'epoch': epoch_i + 1,
          'Training Loss': avg_train_loss,
          'Valid. Loss': avg_val_loss,
          'Valid. Accur.': avg_val_accuracy,
          'Training Time': train_time,
          'Validation Time': validation_time
      }
  )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======= EPOCH 1 / 3 ======
Training...
   Batch    40 of 293047. Elapsed: 0:00:42 Loss: 0.6315  Average_Train_Loss: 0.0337
   Batch    80 of 293047. Elapsed: 0:01:25 Loss: 0.8407  Average_Train_Loss: 0.0302
   Batch   120 of 293047. Elapsed: 0:02:08 Loss: 0.2523  Average_Train_Loss: 0.0275
   Batch   160 of 293047. Elapsed: 0:02:50 Loss: 0.6265  Average_Train_Loss: 0.0262
   Batch   200 of 293047. Elapsed: 0:03:32 Loss: 0.6306  Average_Train_Loss: 0.0255
   Batch   240 of 293047. Elapsed: 0:04:14 Loss: 0.4877  Average_Train_Loss: 0.0247
   Batch   280 of 293047. Elapsed: 0:04:57 Loss: 0.5659  Average_Train_Loss: 0.0238
   Batch   320 of 293047. Elapsed: 0:05:39 Loss: 0.5523  Average_Train_Loss: 0.0233
   Batch   360 of 293047. Elapsed: 0:06:21 Loss: 0.5753  Average_Train_Loss: 0.0226
   Batch   400 of 293047. Elapsed: 0:07:03 Loss: 0.6183  Average_Train_Loss: 0.0221
   Batch   440 of 293047. Elapsed: 0:07:46 Loss: 0.5305  Average_Train_Loss: 0.0215
   Batch   480 of 293047. Elapsed: 0

In [23]:
# Display floats with two decimal places.
pd.set_option('display.precision', 4)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap (doesn't seem to work in Colab).
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.4262,0.3591,0.8925,2:41:23,0:02:49
2,0.3310,0.3280,0.8978,2:41:26,0:02:49
3,0.2677,0.3262,0.8978,2:41:25,0:02:49


# Test

In [69]:
sentences= X_test['Описание заявки'].values
labels = y_test['TARGET_NUM'].values

input_ids = []
attention_masks = []

for sent in tqdm(sentences):
  encoded_dict = tokenizer.encode_plus(
      sent,
      add_special_tokens = True,
      max_length = 256,
      pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt',
  )
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

test_data = TensorDataset(input_ids, attention_masks, labels)


  0%|          | 0/166100 [00:00<?, ?it/s]

/home/daozerova/.conda/envs/fl/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [70]:
prediction_sampler = SequentialSampler(test_data)
prediction_dataloader = DataLoader(test_data, sampler=prediction_sampler, batch_size=batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(test_data)))

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in tqdm(prediction_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids,
                     token_type_ids=None,
                     attention_mask=b_input_mask,
                     return_dict=True)

    logits = result.logits

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 166,100 test sentences...


  0%|          | 0/5191 [00:00<?, ?it/s]

    DONE.


In [71]:
# Combine the results across all batches.
concat_predictions = np.concatenate(predictions, axis=0)
concat_predictions

array([[-0.21765205, -1.5519723 , -1.4594034 , ..., -1.2515924 ,
         9.4910145 ,  2.5111623 ],
       [-0.9524071 , -2.7549949 ,  0.518867  , ...,  0.08588488,
         9.839141  ,  1.436584  ],
       [-2.3296    ,  3.281514  , -2.017745  , ..., -2.3463485 ,
         4.526649  ,  4.094553  ],
       ...,
       [-0.97943366,  1.8491099 , -2.7189112 , ..., -0.48699787,
         5.6260653 ,  2.4752731 ],
       [-0.24861492, -3.3224757 , -1.2537609 , ...,  4.329068  ,
         6.3773365 , -1.284208  ],
       [-0.7388959 , -1.6368381 , -1.4841821 , ..., -1.0935892 ,
         6.3063793 ,  5.776569  ]], dtype=float32)

In [72]:
# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(concat_predictions, axis=1).flatten()
flat_predictions

array([14, 14, 14, ..., 14, 14, 14])

In [73]:
from scipy.special import softmax

concat_predictions = softmax(pd.DataFrame(concat_predictions), axis=1)
concat_predictions.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
155638,7.7137e-06,3.2290e-06,3.8565e-06,3.5297e-06,1.3805e-06,0.0004,4.3092e-06,2.5522e-05,8.5211e-06,2.1071e-06,3.6326e-06,3.7286e-06,9.1308e-07,8.0967e-06,0.9995,2.4286e-05
12078,6.8472e-04,5.2171e-02,2.2865e-04,1.1558e-03,3.8404e-03,0.0314,2.4260e-02,3.7655e-03,9.8176e-03,2.9695e-02,5.5119e-02,2.8927e-02,1.1294e-04,2.8730e-03,0.7278,2.8113e-02
10761,1.5488e-04,1.7047e-05,5.2372e-05,6.0910e-05,5.2294e-05,0.0040,2.9302e-04,1.1622e-01,1.7452e-04,2.3764e-04,5.2564e-05,6.3095e-05,1.6256e-05,6.7787e-04,0.8777,1.8853e-04
77513,5.7679e-05,1.5482e-05,1.5753e-05,3.1375e-05,5.4983e-06,0.0609,4.2511e-05,1.2974e-04,4.5082e-05,2.4720e-05,1.7546e-05,2.3985e-05,5.6981e-06,1.3446e-04,0.9384,8.3589e-05
145313,3.9466e-03,5.4008e-04,3.7314e-04,1.4591e-04,7.1520e-05,0.1271,1.2529e-02,8.9271e-03,1.3939e-04,1.0364e-03,9.8028e-05,3.3749e-04,5.9346e-05,4.2346e-04,0.8442,1.6110e-04


In [75]:
# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)
flat_true_labels

array([14, 14, 14, ..., 14, 13, 14])

In [76]:
from sklearn.metrics import f1_score

# Calculate the F1
f1 = f1_score(flat_true_labels, flat_predictions, average='micro')
f2 = f1_score(flat_true_labels, flat_predictions, average='macro')
f3 = f1_score(flat_true_labels, flat_predictions, average='weighted')

print('F1 Score micro: %.3f' % f1)
print('F1 Score macro: %.3f' % f2)
print('F1 Score weighted: %.3f' % f3)

F1 Score micro: 0.896
F1 Score macro: 0.578
F1 Score weighted: 0.889


In [77]:
flat_true_labels, flat_predictions, len(flat_true_labels), len(flat_predictions)

(array([14, 14, 14, ..., 14, 13, 14]),
 array([14, 14, 14, ..., 14, 14, 14]),
 166100,
 166100)

## Model Report ##

In [78]:
def get_optimal_threshold(
    y_test, y_test_pred, 
    metric_thr = 0.9, 
    metric_fun = precision_score, # average_precision_score,
    n_q = 100, start = .01, stop = .99
):
    quantiles = np.quantile(y_test_pred, np.linspace(.1, .99, num=n_q))
    optimal_threshold = quantiles[-1]
    metric_history = pd.Series(index=quantiles, dtype=np.float64)
    for threshold in quantiles:
        y_test_labels = (y_test_pred > threshold).astype(int)
        metric = metric_fun(y_test, y_test_labels)
        metric_history[threshold] = metric
        if metric >= metric_thr:
            optimal_threshold = threshold
            return optimal_threshold, quantiles, metric_history
    
    return optimal_threshold, quantiles, metric_history

In [79]:
def get_optimal_thresholds(
    y_test_bin, y_test_pred_bin, 
    y_test_multi, y_test_pred_multi,
    metric_fun = precision_score,
    bin_thr   = .90,
    multi_thr = .8,
):
    optimal_threshold_bin, quantiles_bin, metric_history_bin =\
    get_optimal_threshold(
        1 - y_test_bin, 
        1 - y_test_pred_bin, 
        metric_fun = metric_fun,
        metric_thr = bin_thr,
    )

    optimal_threshold_multi, quantiles_multi, metric_history_multi =\
    dict(), dict(), dict()

    for label in y_test_pred_multi.columns:
        _optimal_threshold, _quantiles, _metric_history =\
        get_optimal_threshold(
            (y_test_multi == label).astype(int), 
            y_test_pred_multi.loc[y_test_multi.index, label], 
            metric_fun = metric_fun,
            metric_thr = multi_thr,
        )
        optimal_threshold_multi[label] = _optimal_threshold
        quantiles_multi[label]         = _quantiles
        metric_history_multi[label]    = _metric_history

    optimal_threshold_multi = pd.Series(optimal_threshold_multi)
    
    return optimal_threshold_bin, quantiles_bin, metric_history_bin,\
           optimal_threshold_multi, quantiles_multi, metric_history_multi

In [80]:
def bin_multi_classification_report(
    y_test, y_test_bin, y_test_multi, 
    y_test_pred_bin, y_test_pred_multi,
    optimal_threshold_bin,
    optimal_threshold_multi,
    display_results = True,
):
    #### Применение порогов
    y_test_labels_bin = 1 - (1 - y_test_pred_bin > optimal_threshold_bin)
    '''
    y_test_labels_multi = y_test_pred_multi.ge(optimal_threshold_multi, axis=1)\
                          .mul(optimal_threshold_multi.index + 1).add(-1).max(axis=1)
    '''
    y_test_labels_multi = pd.Series(np.argmax(y_test_pred_multi.ge(optimal_threshold_multi, axis=1)\
                      .mul(y_test_pred_multi).values, axis=1), index=y_test_pred_multi.index)

    y_test_labels_multi.loc[~y_test_pred_multi.ge(optimal_threshold_multi, axis=1).any(axis=1)] = -1

    
    mask = (y_test_labels_multi != -1) & y_test_labels_multi.index.isin(y_test_multi.index)
    #print( y_test_labels_multi,  y_test_labels_multi.index, y_test_labels_multi[mask].index)

    #### Оценка бинарной классификации
    precision, recall, f1_score, support = metrics.precision_recall_fscore_support(y_test_bin, y_test_labels_bin)
    df_metrics_bin = pd.DataFrame({
        'Precision': precision,
        'Recall': recall,
        'F1_Score': f1_score,
        'Target count': support,
        'Predicted count': y_test_labels_bin.value_counts().sort_index().values,
    }, index = [f'Class {i}' for i in range(precision.shape[0])])
    df_metrics_bin = df_metrics_bin.rename_axis('Binary classification', axis=1)
    
    if display_results:
        display(df_metrics_bin)

    #### Мультитаргет классификации
    mask = (y_test_labels_multi != -1) & y_test_labels_multi.index.isin(y_test_multi.index)

    precision, recall, f1_score, support = metrics.precision_recall_fscore_support(
        y_test_multi[mask], 
        y_test_labels_multi[mask],
        zero_division = 0
    )
    
    df_metrics_multi = pd.DataFrame({
        'Precision': precision,
        'Recall': recall,
        'F1_Score': f1_score,
        'Target count': support,
        'Predicted count': y_test_labels_multi[mask].value_counts().sort_index().reindex(np.arange(support.shape[0])).fillna(0).astype(int).values,
    }, index = [f'Class {i}'for i in range(precision.shape[0])])
    df_metrics_multi = df_metrics_multi.rename_axis('Multi classification', axis=1)
    
    if display_results:
        display(df_metrics_multi)

    #### Оценка бинарной + мультитаргет классификации
    y_test_labels_multi = y_test_pred_multi.ge(optimal_threshold_multi, axis=1)\
                          .mul(optimal_threshold_multi.index + 1).add(-1).max(axis=1)

    mask = (y_test_labels_multi != -1) & (y_test_labels_bin == 0)

    precision, recall, f1_score, support = metrics.precision_recall_fscore_support(
        y_test[mask], 
        y_test_labels_multi[mask].replace({-1: 14, 14: 15}),
        zero_division = 0
    )

    df_metrics_bin_multi = pd.DataFrame({
        'Precision': precision,
        'Recall': recall,
        'F1_Score': f1_score,
        'Target count': support,
        'Predicted count': y_test_labels_multi[mask].replace({-1: 14, 14: 15}).value_counts()\
                           .sort_index().reindex(np.arange(support.shape[0])).fillna(0).astype(int).values,
    }, index = [f'Class {i}'for i in range(precision.shape[0])])
    df_metrics_bin_multi = df_metrics_bin_multi.rename_axis('Binary + Multi classification', axis=1)
    
    if display_results:
        display(df_metrics_bin_multi)

    #### Оценка бинарной + мультитаргет классификации в общем
    y_test_labels_multi_all = y_test_labels_multi.replace({-1: 14, 14: 15})
    y_test_labels_multi_all.loc[y_test_labels_bin == 1] = 14

    precision, recall, f1_score, support = metrics.precision_recall_fscore_support(
        y_test, 
        y_test_labels_multi_all,
        zero_division = 0
    )
    df_metrics_multi_all = pd.DataFrame({
        'Precision': precision,
        'Recall': recall,
        'F1_Score': f1_score,
        'Target count': support,
        'Predicted count': y_test_labels_multi_all.value_counts()\
                           .sort_index().reindex(np.arange(16)).fillna(0).astype(int).values,
    }, index = [f'Class {i}'for i in range(precision.shape[0])])
    df_metrics_multi_all = df_metrics_multi_all.rename_axis('Total classification', axis=1)

    if display_results:
        display(df_metrics_multi_all)

    #### Эффект от модели
    df_model_effect = pd.Series(name = 'Показатель', dtype = np.float64)
    df_model_effect.loc['Всего заявок, шт.'] = df_metrics_multi_all.loc[:, 'Target count'].sum()
    df_model_effect.loc['Правильных заявок, шт.'] = df_metrics_multi_all.loc['Class 14', 'Target count']
    df_model_effect.loc['Перенаправленных заявок, шт.'] = df_metrics_multi_all.loc[
        df_metrics_multi_all.index != 'Class 14', 
        'Target count'
    ].sum()
    df_model_effect.loc['Перенаправленных заявок, %'] =\
    df_model_effect.loc['Перенаправленных заявок, шт.'] / df_model_effect.loc['Всего заявок, шт.'] * 100
    df_model_effect.loc['Перенаправленных заявок по модели (всего), шт.'] = df_metrics_multi_all.loc[
        df_metrics_multi_all.index != 'Class 14', 
        'Predicted count'
    ].sum()
    df_model_effect.loc['Перенаправленных заявок по модели (всего), %'] =\
    df_model_effect.loc['Перенаправленных заявок по модели (всего), шт.'] / df_model_effect.loc['Всего заявок, шт.'] * 100
    df_model_effect.loc['Перенаправленных заявок по модели (правильно), шт.'] = df_metrics_multi_all.loc[
        df_metrics_multi_all.index != 'Class 14', 
        'Predicted count'
    ].mul(df_metrics_multi_all.loc[
        df_metrics_multi_all.index != 'Class 14', 
        'Precision'
    ]).sum()
    df_model_effect.loc['Перенаправленных заявок по модели (правильно), %'] =\
    df_model_effect.loc['Перенаправленных заявок по модели (правильно), шт.'] / df_model_effect.loc['Всего заявок, шт.'] * 100
    df_model_effect.loc['Снижение неправильных заявок на, %'] =\
    ( df_model_effect.loc['Перенаправленных заявок по модели (правильно), шт.'] ) / df_model_effect.loc['Перенаправленных заявок, шт.'] * 100 #- df_metrics_bin_multi.loc['Class 14', 'Target count'] 
    df_model_effect.loc['Снижение неправильных заявок (c учетом ошибок) на, %'] =\
    ( df_model_effect.loc['Перенаправленных заявок по модели (правильно), шт.'] - df_metrics_bin_multi.loc['Class 14', 'Target count'] ) / df_model_effect.loc['Перенаправленных заявок, шт.'] * 100 

    df_model_effect = df_model_effect.round(2).to_frame().rename_axis('Эффект от модели', axis=1)
    #print(df_model_effect.loc['Снижение неправильных заявок (c учетом ошибок) на, %'])
    #print(type(df_model_effect.loc['Снижение неправильных заявок (c учетом ошибок) на, %']))
    a = df_model_effect.values[-1][0]
    #print(a, type(a))
    if display_results:
        display(df_model_effect)
    
    return df_metrics_bin, df_metrics_multi, df_metrics_bin_multi, df_metrics_multi_all, df_model_effect, a

In [81]:
y_test = pd.Series(flat_true_labels)

In [82]:
#Target
y_test_bin = pd.Series(flat_true_labels).eq(14).astype(int)
y_test_multi = y_test[y_test != 14].replace({15:14})
y_test = pd.Series(flat_true_labels)
y_test_bin, y_test_multi, y_test

(0         1
 1         1
 2         1
 3         1
 4         1
          ..
 166095    1
 166096    1
 166097    1
 166098    0
 166099    1
 Length: 166100, dtype: int64,
 8         11
 10         5
 17         5
 24         2
 31         5
           ..
 166060     5
 166061    14
 166065    13
 166094    11
 166098    13
 Length: 28224, dtype: int64,
 0         14
 1         14
 2         14
 3         14
 4         14
           ..
 166095    14
 166096    14
 166097    14
 166098    13
 166099    14
 Length: 166100, dtype: int64)

In [83]:
#Predictions
y_test_pred_multi = concat_predictions
#pd.DataFrame(
#    best_lgb_multi.predict(X_test_features), 
#    index = X_test_features.index
#)

y_test_pred_bin = y_test_pred_multi.loc[:, 14]

y_test_pred_multi = y_test_pred_multi.drop(columns=[14]).rename(columns={15: 14})
y_test_pred_multi = y_test_pred_multi.div(1 - y_test_pred_bin, axis=0)

In [87]:
max_perc = 0
save_i, save_j  = 0, 0
for i in range(65, 85):
    for j in range(60, 85):
        optimal_threshold_bin, quantiles_bin, metric_history_bin,\
        optimal_threshold_multi, quantiles_multi, metric_history_multi =\
        get_optimal_thresholds(
            y_test_bin, y_test_pred_bin, 
            y_test_multi, y_test_pred_multi,
            metric_fun = precision_score,
            bin_thr   = i / 100, #0.77, #.75,
            multi_thr = j / 100, #0.8, #.7,
        )
        _, _, _, _, _, a =\
        bin_multi_classification_report(
            y_test, y_test_bin, y_test_multi, 
            y_test_pred_bin, y_test_pred_multi,
            optimal_threshold_bin,
            optimal_threshold_multi,
            display_results = False,
        )
        if a > max_perc:
            max_perc = a
            save_i, save_j = i, j
            
print(max_perc, save_i, save_j)

37.84 83 64


In [90]:
optimal_threshold_bin, quantiles_bin, metric_history_bin,\
optimal_threshold_multi, quantiles_multi, metric_history_multi =\
get_optimal_thresholds(
    y_test_bin, y_test_pred_bin, 
    y_test_multi, y_test_pred_multi,
    metric_fun = precision_score,
    bin_thr   = save_i / 100, #0.77, #.75,
    multi_thr = save_j / 100, #0.8, #.7,
)

In [91]:
df_metrics_bin, df_metrics_multi,\
df_metrics_bin_multi, df_metrics_multi_all,\
df_model_effect, _ =\
bin_multi_classification_report(
    y_test, y_test_bin, y_test_multi, 
    y_test_pred_bin, y_test_pred_multi,
    optimal_threshold_bin,
    optimal_threshold_multi,
    display_results = True,
)

Binary classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.8462,0.6318,0.7235,28224,21073
Class 1,0.9284,0.9765,0.9518,137876,145027


Multi classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.7551,0.8055,0.7795,1095,1168
Class 1,0.7099,0.6881,0.6988,1273,1234
Class 2,0.8736,0.9487,0.9096,3315,3600
Class 3,0.6591,0.6273,0.6428,829,789
Class 4,0.6170,0.5472,0.5800,318,282
Class 5,0.7202,0.7206,0.7204,3644,3646
Class 6,0.6996,0.4422,0.5419,848,536
Class 7,0.7326,0.8102,0.7695,2134,2360
Class 8,0.6872,0.5156,0.5892,1730,1298
Class 9,0.6493,0.4158,0.5069,837,536


Binary + Multi classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.7069,0.7235,0.7151,510,522
Class 1,0.6208,0.7282,0.6702,780,915
Class 2,0.7944,0.9076,0.8472,3107,3550
Class 3,0.5286,0.6332,0.5762,379,454
Class 4,0.5936,0.6250,0.6089,208,219
Class 5,0.6780,0.7074,0.6924,1938,2022
Class 6,0.5299,0.4562,0.4903,388,334
Class 7,0.6898,0.8762,0.7719,1284,1631
Class 8,0.5620,0.5587,0.5604,1022,1016
Class 9,0.5318,0.4871,0.5084,464,425


Total classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.7069,0.3361,0.4556,1098,522
Class 1,0.6208,0.4195,0.5007,1354,915
Class 2,0.7944,0.8507,0.8216,3315,3550
Class 3,0.5286,0.2634,0.3516,911,454
Class 4,0.5936,0.3714,0.4569,350,219
Class 5,0.6780,0.3667,0.4760,3739,2022
Class 6,0.5299,0.1879,0.2774,942,334
Class 7,0.6898,0.5151,0.5898,2184,1631
Class 8,0.5620,0.3028,0.3935,1886,1016
Class 9,0.5318,0.2042,0.2950,1107,425


Эффект от модели,Показатель
"Всего заявок, шт.",166100.00
"Правильных заявок, шт.",137876.00
"Перенаправленных заявок, шт.",28224.00
"Перенаправленных заявок, %",16.99
"Перенаправленных заявок по модели (всего), шт.",20613.00
"Перенаправленных заявок по модели (всего), %",12.41
"Перенаправленных заявок по модели (правильно), шт.",13814.00
"Перенаправленных заявок по модели (правильно), %",8.32
"Снижение неправильных заявок на, %",48.94
"Снижение неправильных заявок (c учетом ошибок) на, %",37.84


In [84]:
optimal_threshold_bin, quantiles_bin, metric_history_bin,\
optimal_threshold_multi, quantiles_multi, metric_history_multi =\
get_optimal_thresholds(
    y_test_bin, y_test_pred_bin, 
    y_test_multi, y_test_pred_multi,
    metric_fun = precision_score,
    bin_thr   = 0.8, #save_i, #0.77, #.75,
    multi_thr = 0.75, #save_j, #0.8, #.7,
)

In [85]:
df_metrics_bin, df_metrics_multi,\
df_metrics_bin_multi, df_metrics_multi_all,\
df_model_effect, _ =\
bin_multi_classification_report(
    y_test, y_test_bin, y_test_multi, 
    y_test_pred_bin, y_test_pred_multi,
    optimal_threshold_bin,
    optimal_threshold_multi,
    display_results = True,
)

Binary classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.8097,0.6902,0.7452,28224,24060
Class 1,0.9384,0.9668,0.9524,137876,142040


Multi classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.8013,0.8227,0.8119,1015,1042
Class 1,0.7644,0.8227,0.7925,970,1044
Class 2,0.8405,0.9665,0.8991,3315,3812
Class 3,0.7668,0.6861,0.7242,599,536
Class 4,0.6170,0.6960,0.6541,250,282
Class 5,0.7691,0.7721,0.7706,3278,3291
Class 6,0.8021,0.3847,0.5200,590,283
Class 7,0.7997,0.8511,0.8246,1928,2052
Class 8,0.7886,0.4917,0.6057,1267,790
Class 9,0.7491,0.5060,0.6040,419,283


Binary + Multi classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.6270,0.8031,0.7042,584,748
Class 1,0.5973,0.8315,0.6952,742,1033
Class 2,0.7607,0.9477,0.8440,3193,3978
Class 3,0.5274,0.6897,0.5977,377,493
Class 4,0.5314,0.7273,0.6141,198,271
Class 5,0.6357,0.7587,0.6917,1964,2344
Class 6,0.5261,0.4393,0.4788,321,268
Class 7,0.7045,0.8898,0.7864,1289,1628
Class 8,0.5753,0.5533,0.5640,967,930
Class 9,0.5437,0.5559,0.5498,313,320


Total classification,Precision,Recall,F1_Score,Target count,Predicted count
Class 0,0.6270,0.4271,0.5081,1098,748
Class 1,0.5973,0.4557,0.5170,1354,1033
Class 2,0.7607,0.9128,0.8298,3315,3978
Class 3,0.5274,0.2854,0.3704,911,493
Class 4,0.5314,0.4114,0.4638,350,271
Class 5,0.6357,0.3985,0.4899,3739,2344
Class 6,0.5261,0.1497,0.2331,942,268
Class 7,0.7045,0.5252,0.6018,2184,1628
Class 8,0.5753,0.2837,0.3800,1886,930
Class 9,0.5437,0.1572,0.2439,1107,320


Эффект от модели,Показатель
"Всего заявок, шт.",166100.00
"Правильных заявок, шт.",137876.00
"Перенаправленных заявок, шт.",28224.00
"Перенаправленных заявок, %",16.99
"Перенаправленных заявок по модели (всего), шт.",20846.00
"Перенаправленных заявок по модели (всего), %",12.55
"Перенаправленных заявок по модели (правильно), шт.",14081.00
"Перенаправленных заявок по модели (правильно), %",8.48
"Снижение неправильных заявок на, %",49.89
"Снижение неправильных заявок (c учетом ошибок) на, %",36.53


In [ ]:
pd.DataFrame(y_test, columns=['0']).to_parquet('bert_16/y_test.parquet')
pd.DataFrame(y_test_bin, columns=['0']).to_parquet('bert_16/y_test_bin.parquet')
pd.DataFrame(y_test_multi, columns=['0']).to_parquet('bert_16/y_test_multi.parquet')

pd.DataFrame(y_test_pred_bin, columns=['0']).to_parquet('bert_16/y_test_pred_bin.parquet')
pd.DataFrame(y_test_pred_multi, columns=['0']).to_parquet('bert_16/y_test_pred_multi.parquet')

In [6]:
total_class = pd.read_excel('total_class.xlsx').drop(['Unnamed: 0'], axis=1)
total_class

,Total classification,Precision,Recall,F1_Score,Target count,Predicted count
0,DebitCardsSupport,0.7069,0.3361,0.4556,1098,522
1,MB.oplata_dbo,0.6208,0.4195,0.5007,1354,915
2,Авторизация. Неактуальная,0.7944,0.8507,0.8216,3315,3550
3,Антифрод,0.5286,0.2634,0.3516,911,454
4,Возврат средств,0.5936,0.3714,0.4569,350,219
5,Другое: неправильная заявка,0.6780,0.3667,0.4760,3739,2022
6,МТС Деньги ВЕБ,0.5299,0.1879,0.2774,942,334
7,Отдел активно-пассивных операций,0.6898,0.5151,0.5898,2184,1631
8,Отдел поддержки карточных технологий,0.5620,0.3028,0.3935,1886,1016
9,Переводы между своими счетами,0.5318,0.2042,0.2950,1107,425


In [15]:
166100.00 - (142040 + 20846)

3214.0